## DataSet

In [1]:
from dataloader import get_dataloader
import config as CFG
from tokenizer import get_tokenizer

In [2]:

tokenizer = get_tokenizer(CFG.text_model_name)
dataloader_train = get_dataloader(tokenizer=tokenizer,batch_size=CFG.batch_size,shuffle=CFG.shuffle,num_workers=CFG.num_workers,split=CFG.split)

Using downloaded and verified file: ./flickr30k/flickr30k_train.json


## Model

In [3]:
from models import CLIPModel
from losses import CLIPLoss

## Training

In [4]:
import wandb

In [5]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: carrelv. Use `wandb login --relogin` to force relogin


True

In [6]:
wandb.init(project="master_test_1",
           config={
               "batch_size": 2,
               "learning_rate": 1e-3,
               "dataset": "flickr30k",
           })

In [6]:
from training import get_lr 
import torch
import itertools

In [7]:
model = CLIPModel()

Some weights of ViTModel were not initialized from the model checkpoint at facebook/dino-vits16 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the

In [12]:
params = [
            {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
            {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
            {"params": itertools.chain(
                model.image_projection.parameters(), model.text_projection.parameters()
            ), "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
        ]

In [13]:
optimizer = torch.optim.AdamW(params, weight_decay=0.)




In [14]:
lr=get_lr(optimizer)

In [15]:
lr

0.0001